In [1]:
import os
from skimage.transform import resize
import skimage.io as io 
import numpy as np
import matplotlib.pyplot as plt

from func.tool import get_fname, check_dir
from func.plot import plt_result

In [2]:
FLICKR = 'data/ori/tesri/'
model = 'model/Unet_5comps/5comps_model/'
TARGET = os.path.join(model,'Predict_mask')

SUBDIR  = ['A_rmbg_01_Body', 'A_rmbg_02_Left_fore','A_rmbg_03_Right_fore', 'A_rmbg_04_Left_hind', 'A_rmbg_05_Right_hind']
RMBG      = 'data/processed/finaluse/A_rmbg_00_img/'
BODY      = os.path.join(TARGET, SUBDIR[0])
UPLEFT    = os.path.join(TARGET, SUBDIR[1])
UPRIGHT   = os.path.join(TARGET, SUBDIR[2])
DOWNLEFT  = os.path.join(TARGET, SUBDIR[3])
DOWNRIGHT = os.path.join(TARGET, SUBDIR[4])

SAVEDIR = 'data/processed/finaluse/'

In [3]:
target = os.listdir(FLICKR)
flickr    = [os.path.join(FLICKR,    get_fname(i)+'.jpg') for i in target]
rmbg      = [os.path.join(RMBG,      get_fname(i)+'.png') for i in target]
body      = [os.path.join(BODY,      get_fname(i)+'.png') for i in target]
upleft    = [os.path.join(UPLEFT,    get_fname(i)+'.png') for i in target]
upright   = [os.path.join(UPRIGHT,   get_fname(i)+'.png') for i in target]
downleft  = [os.path.join(DOWNLEFT,  get_fname(i)+'.png') for i in target]
downright = [os.path.join(DOWNRIGHT, get_fname(i)+'.png') for i in target]

In [4]:
def img_0to1(x):
    if x.max() >1:
        x = x/255
    else:
        x = x
    return x

In [5]:
def img_thrshld(x, thrshld=0.5):
    x_ = x.copy()
    x_[x_>=0.5] = 1
    x_[x_<0.5]  =0
    return x_

In [6]:
def img1_img3(x):
    x3 = np.stack((x,x,x),axis=2)
    return x3

In [7]:
def masktoimg(original,x,t_x):
    comp = x*original+t_x
    plt.imshow(comp, cmap = 'gray')
    #plt.show()
    plt.close()
    return comp

In [8]:
for i in range(len(target)):
    p_Oflickr = img_0to1(io.imread(flickr[i]))
    p_flickr = resize(p_Oflickr, (256,256))
    p_rmbg      = img_0to1(io.imread(rmbg[i]))
    p_body      = img1_img3(img_thrshld(img_0to1((io.imread(body[i])))))
    p_upleft    = img1_img3(img_thrshld(img_0to1((io.imread(upleft[i])))))
    p_upright   = img1_img3(img_thrshld(img_0to1((io.imread(upright[i])))))
    p_downleft  = img1_img3(img_thrshld(img_0to1((io.imread(downleft[i])))))
    p_downright = img1_img3(img_thrshld(img_0to1((io.imread(downright[i])))))
    t_body      = 1-p_body
    t_upleft    = 1-p_upleft
    t_upright   = 1-p_upright
    t_downleft  = 1-p_downleft
    t_downright = 1-p_downright

    #plt.imshow(p_flickr)
    #plt.show()
    b =masktoimg(p_flickr, p_body,      t_body)
    ul=masktoimg(p_flickr, p_upleft,    t_upleft)
    ur=masktoimg(p_flickr, p_upright,   t_upright)
    dl=masktoimg(p_flickr, p_downleft,  t_downleft)
    dr=masktoimg(p_flickr, p_downright, t_downright)
    OUTPUT_DIR = 'data/processed/finaluse/'
    
    fname = get_fname(flickr[i])
    p_img = [p_flickr,p_rmbg,b,ul,ur,dl,dr]
    #p_title= ['TESRI-dataset\n%s' %(fname), 'rmbg']+SUBDIR
    p_title = [('') for i in range(len(p_img)) ]
    fig = plt_result(p_img, p_title)
    
    save_to_check = os.path.join(model, 'CHECKING')
    check_dir(save_to_check)
    fig.savefig(os.path.join(save_to_check, fname+'.png'), dpi=100, format='png',bbox_inches='tight')
    
    for i in range(5):
        save_to_5comp = os.path.join(SAVEDIR, SUBDIR[i])
        check_dir(save_to_5comp)
        io.imsave(os.path.join(save_to_5comp, fname+'.png'), p_img[2+i])

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: data/processed/finaluse/A_rmbg_04_Left_hind/V04-20130411-095.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: data/processed/finaluse/A_rmbg_05_Right_hind/V04-20130411-095.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: data/processed/finaluse/A_rmbg_02_Left_fore/V02-20130407-057.png is a low contrast image
  warn